In [1]:
import requests
import rasterio
from rasterio.io import MemoryFile

# --- Define bounding box in lat/long (EPSG:4326) ---
lon_min = 11.668473
lat_min = -18.042082
lon_max = 24.082117
lat_max = -4.372591

# Soil properties and depths of interest
properties = ["sand", "silt", "clay", "phh2o"]
depths = ["0-5cm", "5-15cm", "15-30cm"]

# Output directory
out_dir = "./soilgrids_downloads/"

# Base URL for SoilGrids
base_url = "https://rest.soilgrids.org/soilgrids/v2.0/properties/query"

# Loop through all combinations
for prop in properties:
    for depth in depths:
        url = (
            f"{base_url}"
            f"?bbox={lon_min},{lat_min},{lon_max},{lat_max}"
            f"&property={prop}"
            f"&depth={depth}"
            f"&value=mean"   # use 'mean' prediction (other options: uncertainty, q0.05, q0.5, q0.95)
            f"&format=geotiff"
        )

        print(f"Downloading {prop} at {depth} ...")

        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with MemoryFile(r.content) as memfile:
                with memfile.open() as dataset:
                    # Save to disk
                    out_path = f"{out_dir}{prop}_{depth}.tif"
                    with rasterio.open(out_path, "w", **dataset.profile) as dst:
                        dst.write(dataset.read())
            print(f"✔ Saved to {out_path}")
        else:
            print(f"✘ Failed to download {prop} at {depth}: {r.status_code}")


ConnectionError: HTTPSConnectionPool(host='rest.soilgrids.org', port=443): Max retries exceeded with url: /soilgrids/v2.0/properties/query?bbox=11.668473,-18.042082,24.082117,-4.372591&property=sand&depth=0-5cm&value=mean&format=geotiff (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x120477910>: Failed to resolve 'rest.soilgrids.org' ([Errno 8] nodename nor servname provided, or not known)"))